# Build DIM Tables

After doing some work on the key boxscore and scoring data, I want to build out the framework for dimensional tables that could be a part of these datasets:

- dim_team
- dim_game
- dim_date
- dim_schedule
- dim_score_type
- dim_sportsbook

Using Rapid API Link: https://rapidapi.com/tank01/api/tank01-nfl-live-in-game-real-time-statistics-nfl

The free tier offers 100 calls per day

## Imports

In [2]:
import pandas as pd
import json
import os
import requests

from dotenv import load_dotenv

## Get API Credentials

In [3]:
#Get API Key and host
load_dotenv()
api_token = os.getenv('nfl_api_key')
api_host = os.getenv('rapid_api_host')

## Team Data

This data will come from the "Get NFL Teams" endpoint in the Rapid API.

### Extract

In [4]:
#API Endpoint for NFL Team Info
url = "https://tank01-nfl-live-in-game-real-time-statistics-nfl.p.rapidapi.com/getNFLTeams"

In [5]:
#rapidapi headers
headers = {
	"X-RapidAPI-Key": "{key}".format(key=api_token),
	"X-RapidAPI-Host": "{host}".format(host=api_host)
}

In [6]:
#pull in only main team info, none of the rosters, schedules, stats, etc.
querystring = {"rosters":"false","schedules":"false","topPerformers":"false","teamStats":"false"}

In [7]:
#get team data
response = requests.get(url, headers=headers, params=querystring)

In [11]:
#export data file for reference
with open('../data/team_data.json', 'w') as file:
    json.dump(response.json(), file)

### Transform

In [13]:
#drill into body of api response
teams = response.json()['body']

In [14]:
teams[0]

{'teamAbv': 'ARI',
 'teamCity': 'Arizona',
 'currentStreak': {'result': 'W', 'length': '1'},
 'loss': '2',
 'teamName': 'Cardinals',
 'nflComLogo1': 'https://static.www.nfl.com/image/private/f_auto/league/u9fltoslqdsyao8cpm0k',
 'teamID': '1',
 'tie': '0',
 'pa': '67',
 'pf': '72',
 'espnLogo1': 'https://a.espncdn.com/combiner/i?img=/i/teamlogos/nfl/500/ari.png',
 'wins': '1'}

In [17]:
#save record for each team in list
team_list = []

#create dictionary for team data
for team in teams:
    team_info = {
        "team_id": team['teamID'],
        "team_name_location": team['teamCity'],
        "team_name": team['teamName'],
        "team_abrv": team['teamAbv'],
        "team_logo_link": team['nflComLogo1']
    }

    team_list.append(team_info)

In [20]:
team_list[:2]

[{'team_id': '1',
  'team_name_location': 'Arizona',
  'team_name': 'Cardinals',
  'team_abrv': 'ARI',
  'team_logo_link': 'https://static.www.nfl.com/image/private/f_auto/league/u9fltoslqdsyao8cpm0k'},
 {'team_id': '2',
  'team_name_location': 'Atlanta',
  'team_name': 'Falcons',
  'team_abrv': 'ATL',
  'team_logo_link': 'https://static.www.nfl.com/image/private/f_auto/league/d8m7hzpsbrl6pnqht8op'}]